In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imsave
from skimage.io import imread_collection, imread
from skimage.color import rgb2gray

In [ ]:
DIR = 'imagens_teste/red/*.png'
images = imread_collection(DIR)

fig, ax = plt.subplots(1, 12, figsize=(40,40))
for i, image in enumerate(images):
    ax[i].imshow(image, cmap='gray')

In [ ]:
def gray_255_scale(image):  
    arr = np.zeros(image.shape)
    for i in range(len(arr)):
        for j in range(len(arr[i])):
            arr[i][j] = int(round(image[i,j] * 255))
    return arr

In [ ]:
def homogeneity(image):
    soma = 0
    for i in range(len(image)):
        for j in range(len(image[i])):
            soma+=(image[i][j]/(1+((i-j)**2)))
    
    return soma

def variance(image):
    i_sum = 0
    j_sum = 0

    mean = np.mean(image)

    for i in range(len(image)):
        j_sum = 0
        for j in range(len(image[i])):
            j_sum += ((1-mean)**2)*image[i,j]
        i_sum+=j_sum

    return i_sum

def uniformity(image):
    sum_index_values = 0
    for i in image:
        for j in i:
            sum_index_values += j**2
    return sum_index_values

def entropy(image):
    sum_index_values = 0
    for i in image:
        for j in i:
            if j > 0:
                sum_index_values += j*np.log10(j)
    return -(sum_index_values)

def correlation(image):
    sum_index_values = 0

    for i in range(len(image)):
        for j in range(len(image[i])):
            if image[i,j] != 0:
                sum_index_values += (i - np.mean(image[i])) * (j - np.mean(image[:,j])) * image[i,j]/(np.std(image[i])*np.std(image[:,j]))
    return sum_index_values

def contrast(image):
    sum_index_values = 0
    
    for i in range(len(image)):
        for j in range(len(image[i])):
            sum_index_values += ((i - j) ** 2) * image[i,j]
    return sum_index_values

In [ ]:
def glcm(image, angle, distance, normalize=False):

    lines, columns = image.shape

    glcm_matrix = []
    glcm_lines=0
    glcm_columns=0
    
    if(angle==0):
        for i in range(lines):
            for j in range(columns-distance):
                reference = int(image[i][j])
                neighbour = int(image[i][j+distance])
                if(reference > glcm_lines):
                    glcm_lines = reference
                if(neighbour > glcm_columns):
                    glcm_columns = neighbour         
        for i in range(glcm_lines+1):
            lista = []
            for j in range(glcm_columns+1):
                lista.append(0)
            glcm_matrix.append(lista)

        for i in range(lines):
            for j in range(columns-distance):
                reference = int(image[i][j])
                neighbour = int(image[i][j+distance])
                glcm_matrix[reference][neighbour]+=1
        if normalize == True:
            return (glcm_matrix - np.min(glcm_matrix))/np.ptp(glcm_matrix)
        return glcm_matrix

    if(angle == 90):
        for i in range(distance, lines, 1):
            for j in range(columns):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j])
                if(reference > glcm_lines):
                    glcm_lines = reference
                if(neighbour > glcm_columns):
                    glcm_columns = neighbour

        for i in range(glcm_lines+1):
            lista = []
            for j in range(glcm_columns+1):
                lista.append(0)
            glcm_matrix.append(lista)

        for i in range(distance, lines, 1):
            for j in range(columns):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j])
                glcm_matrix[reference][neighbour]+=1
        if normalize == True:
            return (glcm_matrix - np.min(glcm_matrix))/np.ptp(glcm_matrix)
        return glcm_matrix

    if(angle == 45):
        for i in range(distance, lines, 1):
            for j in range(distance, columns, 1):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j-distance])
                if(reference > glcm_lines):
                    glcm_lines = reference
                if(neighbour > glcm_columns):
                    glcm_columns = neighbour         

        for i in range(glcm_lines+1):
            lista = []
            for j in range(glcm_columns+1):
                lista.append(0)
            glcm_matrix.append(lista)

        for i in range(distance, lines, 1):
            for j in range(distance, columns, 1):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j-distance])
                glcm_matrix[reference][neighbour]+=1
        if normalize == True:
            return (glcm_matrix - np.min(glcm_matrix))/np.ptp(glcm_matrix)
        return glcm_matrix

    if(angle == 135):
        for i in range(distance, lines, 1):
            for j in range(columns-distance):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j+distance])
                if(reference > glcm_lines):
                    glcm_lines = reference
                if(neighbour > glcm_columns):
                    glcm_columns = neighbour

        for i in range(glcm_lines+1):
            lista = []
            for j in range(glcm_columns+1):
                lista.append(0)
            glcm_matrix.append(lista)

        for i in range(distance, lines, 1):
            for j in range(columns-distance):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j+distance])
                glcm_matrix[reference][neighbour]+=1
        if normalize == True:
            return (glcm_matrix - np.min(glcm_matrix))/np.ptp(glcm_matrix)
        return glcm_matrix

In [ ]:
def descritor_glcm(imagem, angle, distance):
    glcm_images = []
    for i in range(len(imagem)):
        print("Imagem {}...".format(i+1))
        gray_im = rgb2gray(imagem[i])
        matriz_glcm = glcm(gray_255_scale(gray_im), angle, distance, normalize=True)
        
        #definição da classe da imagem
        classe = 0
        glcm_images.append([uniformity(np.array(matriz_glcm)), entropy(np.array(matriz_glcm)), contrast(np.array(matriz_glcm)), correlation(np.array(matriz_glcm)), variance(np.array(matriz_glcm)), homogeneity(np.array(matriz_glcm)), classe])
        
    return glcm_images

In [ ]:
#Aplicando com os parâmetros Ângulo = 0 e Distância = 1
columns = ['uniformity', 'entropy', 'contrast', 'correlation', 'variance', 'homogeneity', 'Classe']
data = pd.DataFrame(descritor_glcm(images, 0, 1),columns=columns)
data

In [ ]:
#Aplicando com os parâmetros Ângulo = 45 e Distância = 1
columns = ['uniformity', 'entropy', 'contrast', 'correlation', 'variance', 'homogeneity', 'Classe']
data2 = pd.DataFrame(descritor_glcm(images, 45, 1),columns=columns)
data2

In [ ]:
#Aplicando com os parâmetros Ângulo = 0 e Distância = 2
columns = ['uniformity', 'entropy', 'contrast', 'correlation', 'variance', 'homogeneity', 'Classe']
data3 = pd.DataFrame(descritor_glcm(images, 45, 2),columns=columns)
data3